In [1]:
import pandas as pd
from omop_etl.load import Loader

In [10]:
omop = Loader('z:/Covid19/Covid19_OMOP/new_pipeline/config.yml')

In [11]:
def domain_counts(table):
    
    dimension = table.split('_')[0]

    with omop.engine.connect() as con:
        dom = pd.read_sql('''
        select distinct b.domain_id, count(*) N
        from dbo.{0} a
        join xref.concept b
        on a.{1}_concept_id = b.concept_id
        group by b.domain_id
        '''.format(table, dimension), con)

        return dom

In [13]:
domain_counts('condition_occurrence')

,domain_id,N
0,Metadata,225333
1,Condition,56576414


In [14]:
domain_counts('observation')

,domain_id,N
0,Observation,87570857


In [15]:
domain_counts('measurement')

,domain_id,N
0,Metadata,24882960
1,Measurement,397931257


In [16]:
domain_counts('procedure_occurrence')

,domain_id,N
0,Metadata,1011953
1,Procedure,10997528


In [17]:
domain_counts('drug_exposure')

,domain_id,N
0,Drug,73138066
1,Metadata,270397
